In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [3]:
 matches_df = pd.read_parquet("./esta/parsed/matches.parquet")
 rounds_df = pd.read_parquet("./esta/parsed/rounds.parquet")
 kills_df = pd.read_parquet("./esta/parsed/kills.parquet")
 damages_df = pd.read_parquet("./esta/parsed/damages.parquet") 
 grenades_df = pd.read_parquet("./esta/parsed/grenades.parquet") 
 bomb_events_df = pd.read_parquet("./esta/parsed/bomb_events.parquet") 
 weapon_fires_df = pd.read_parquet("./esta/parsed/weapon_fires.parquet") 
 flashes_df = pd.read_parquet("./esta/parsed/flashes.parquet") 
 frames_df = pd.read_parquet("./esta/parsed/frames.parquet")
 players_df = pd.read_parquet("./esta/parsed/players.parquet") 
 team_frames_df = pd.read_parquet("./esta/parsed/team_frames.parquet") 
 player_frames_df = pd.read_parquet("./esta/parsed/player_frames.parquet") 
 inventory_df = pd.read_parquet("./esta/parsed/inventory.parquet") 
 bomb_location_df = pd.read_parquet("./esta/parsed/bomb_location.parquet") 
 projectiles_df = pd.read_parquet("./esta/parsed/projectiles.parquet") 
 smokes_df = pd.read_parquet("./esta/parsed/smokes.parquet") 
 fires_df = pd.read_parquet("./esta/parsed/fires.parquet") 

image_dim = 1024

In [4]:
with open(".awpy/maps/map-data.json", "r") as f:
    map_data_all = json.load(f)

print(type(map_data_all))

<class 'dict'>


# analysis over single round

In [5]:
selected_map = 'de_mirage'

In [6]:
selected_matches = matches_df[matches_df.map_name == selected_map]
selected_matches.head()

client_name   map_name  tick_rate  playback_ticks  playback_frames_count  \
0   GOTV Demo  de_mirage        127          343251                 342767   
0   GOTV Demo  de_mirage        127          471790                 471564   
0   GOTV Demo  de_mirage        127          426251                 425998   
0   GOTV Demo  de_mirage        127          441259                 440692   
0   GOTV Demo  de_mirage        127          493224                 492014   

   parsed_to_frame_idx                              match_id  \
0               342784  0839a031-0e2c-4c03-b4bc-b5ba05fe669a   
0               471581  0a075b4d-576e-46f2-b944-fe9542faee54   
0               426016  a98ffca7-e968-4a54-9e8f-22f101c2502b   
0               440707  a56115fd-a0ba-4eea-a618-91fd6c57177c   
0               492029  f218743d-5e8b-436e-91bd-b7b3664c56c3   

                                demo_id                  competition_name  \
0  01746483-b3bd-4e4b-a2f2-0de97b17600d            PGL Major Antwerp 2022   
0  04f0fb8f-a4d7-497c-857e-316300fadee3                 IEM Katowice 2022   
0  022866a3-090e-42eb-ab8e-aa553a575ada                 IEM Katowice 2021   
0  015f7c8d-2bc0-4d83-88e7-60648d2b221c  BLAST Premier Fall Showdown 2021   
0  0418eb19-853e-4b56-955c-d3eedf2b7e12   BLAST Premier Spring Final 2021   

                                            hltv_url     match_date  \
0  https://www.hltv.org/matches/2356160/heroic-vs...  1652781600000   
0  https://www.hltv.org/matches/2354381/furia-vs-...  1645124400000   
0  https://www.hltv.org/matches/2346573/faze-vs-f...  1613905200000   
0  https://www.hltv.org/matches/2351973/heroic-vs...  1634034600000   
0  https://www.hltv.org/matches/2349173/natus-vin...  1623853200000   

                                          match_name  
0          heroic-vs-vitality-pgl-major-antwerp-2022  
0                furia-vs-astralis-iem-katowice-2022  
0                    faze-vs-furia-iem-katowice-2021  
0  heroic-vs-poggerz-blast-premier-fall-showdown-...  
0  natus-vincere-vs-faze-blast-premier-spring-fin...

## location based analysis over single round

In [7]:
idx_pf_df = player_frames_df.set_index(["match_id","round_num"])
idx_f_df = flashes_df.set_index(["match_id","round_num"])
idx_sm_df = smokes_df.set_index(["match_id","round_num"])

### transform functions
so data fit the plotting format

In [8]:
def transform_coord(map_data,player_loc):
    result = player_loc.copy()
        
    pos_x = map_data["pos_x"]
    pos_y = map_data["pos_y"]
    scale = map_data["scale"]

    result["x"] = (result["x"] - pos_x) / scale
    result["y"] = image_dim - (pos_y - result["y"]) / scale

    return result

In [9]:
def transform_flash(flashes):
    no_idx_flashes = flashes.reset_index(drop=True)
    flashers = no_idx_flashes[["attacker_x","attacker_y","tick","attacker_side"]]
    flasheds = no_idx_flashes[["player_x","player_y","tick","player_side"]]

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
        flashers.loc[:,"type"] = "flasher"
        flasheds.loc[:,"type"] = "flashed"
        
    flashers.columns = ["x","y","tick","side","type"]
    flasheds.columns = ["x","y","tick","side","type"]

    result = pd.concat([flashers,flasheds]).reset_index(drop=True).sort_values(by=["tick"])
    result["tick"] = result["tick"] - result["tick"].min()

    return result

### plotting functions

In [10]:
def plot_map(map_name,fig_size,fig=None,ax=None):
    fig,ax = plt.subplots(figsize=fig_size)
    map_img = plt.imread(f".awpy/maps/{map_name}.png")
    ax.imshow(map_img,extent=[0,image_dim,0,image_dim])
    ax.set_xlim(0, image_dim)
    ax.set_ylim(0, image_dim)
    ax.set_title(map_name.title())
    return fig,ax

In [11]:
def count_colorbar(fig):
    result = 0
    for ax in fig.axes:
        if "colorbar" in ax.get_label():
            result += 1
    return result

In [12]:
def plot_loc_unicode(player_loc, 
                     gradient_by, 
                     size,
                     color_by=None,
                     color_dict=None,
                     marker_by=None,
                     marker_dict=None,
                     fig=None, ax=None):
    if fig is None and ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))

    selected_col = ["x","y",gradient_by]
    if color_by is not None:
        selected_col.append(color_by)

    if marker_by is not None:
        selected_col.append(marker_by)

    transformed = player_loc.reset_index()[selected_col]
    transformed[color_by] = transformed[color_by].str.lower()

    # Define Unicode marker symbols and colormaps.
    
    side = ['left', 'right']

    n_colorbar = count_colorbar(fig)
    for idx,row in transformed.iterrows():
        color = plt.get_cmap(color_dict[row[color_by]])(row[gradient_by])
        if marker_by is None:
            ax.scatter(row["x"],row["y"],color=color,s=size,alpha=0.5)
        else:
            ax.text(row["x"], row["y"], marker_dict[row[marker_by]], 
                    fontsize=size, color=color, ha='center', va='center',
                   alpha=0.5)


    ax.set_xlabel("X Coordinate (pixels)")
    ax.set_ylabel("Y Coordinate (pixels)")
    for idx,(color_cat,cmap) in enumerate(list(color_dict.items())[:2-n_colorbar]):
        positions = transformed[transformed[color_by] == color_cat]
        dummy_scatter = ax.scatter(positions['x'], positions['y'], c=positions[gradient_by], cmap=cmap,s=0, alpha=.5)
        cbar = fig.colorbar(dummy_scatter, ax=ax, location=side[idx], pad=0.02,
                                    fraction=0.046, shrink=0.6)
        cbar.set_label(f"{color_cat.upper()} {gradient_by.title()}", fontsize=8)
        cbar.ax.tick_params(labelsize=7)
    return fig, ax


In [13]:
def plot_loc_unicode(player_loc, 
                     gradient_by, 
                     size,
                     color_by=None,
                     color_dict=None,
                     default_color="viridis",  # Default colormap when color_by is None
                     alpha=0.5,
                     marker_by=None,
                     marker_dict=None,
                     default_marker="o",       # Default marker when marker_by is None
                     fig=None, ax=None):
    if fig is None and ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))
    selected_col = ["x","y",gradient_by]
    if color_by is not None:
        selected_col.append(color_by)
    if marker_by is not None:
        selected_col.append(marker_by)
    transformed = player_loc.reset_index()[selected_col]
    
    # Create normalization using min and max from the dataframe
    vmin = transformed[gradient_by].min()
    vmax = transformed[gradient_by].max()
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    
    # Process color_by if it exists
    if color_by is not None:
        transformed[color_by] = transformed[color_by].str.lower()
    
    # Define side positions for colorbar
    side = ['left', 'right']
    
    n_colorbar = count_colorbar(fig)
    
    for idx, row in transformed.iterrows():
        if color_by is not None:
            cmap = plt.get_cmap(color_dict[row[color_by]])
        else:
            cmap = plt.get_cmap(default_color)
        
        # Apply normalization
        color_value = norm(row[gradient_by])
        color = cmap(color_value)

        if marker_by is None:
            marker_char = default_marker  # Use default marker
        else:
            marker_char = marker_dict[row[marker_by]]
            
        # Use ax.text for all cases
        ax.text(row["x"], row["y"], marker_char, 
                fontsize=size, color=color, ha='center', va='center',
                alpha=alpha)
        
    ax.set_xlabel("X Coordinate (pixels)")
    ax.set_ylabel("Y Coordinate (pixels)")
    
    # Only create colorbars when color_by is specified
    if color_by is not None:
        for idx, (color_cat, cmap) in enumerate(list(color_dict.items())[:2-n_colorbar]):
            positions = transformed[transformed[color_by] == color_cat]
            dummy_scatter = ax.scatter(positions['x'], positions['y'], 
                                      c=positions[gradient_by], 
                                      cmap=cmap, s=0, alpha=.5,
                                      norm=norm)  # Apply the normalization
            cbar = fig.colorbar(dummy_scatter, ax=ax, location=side[idx], pad=0.02,
                               fraction=0.046, shrink=0.6)
            cbar.set_label(f"{color_cat.upper()} {gradient_by.title()}", fontsize=8)
            cbar.ax.tick_params(labelsize=7)
    
    return fig, ax

### test plotting functions with players' location and flashes' location

In [14]:
selected_match = selected_matches.iloc[0]
selected_round = 10

transformed_flash = transform_flash(idx_f_df.loc[(selected_match.match_id,selected_round)])
transformed_flash = transform_coord(map_data_all[selected_map],transformed_flash)
# transformed_flash = transformed_flash[transformed_flash["type"] == "flasher"]

transformed_loc = transform_coord(map_data_all[selected_map],
                                  idx_pf_df.loc[(selected_match.match_id,selected_round)].iloc[
                                  ::5
                                  ])

transformed_smoke = transform_coord(map_data_all[selected_map],
                                   idx_sm_df.loc[(selected_match.match_id,selected_round)])
side_color = {
    "ct" : "Blues",
    "t" : "Reds"
}

flash_marker = {
    "flasher" : "$\u274B$",
    "flashed" : "⌐■-■",
}
img_fig, img_ax = plot_map(selected_map, (10, 8))

loc_fig, loc_ax = plot_loc_unicode(transformed_loc, 
                                  gradient_by="frame_id", 
                                  size=5,
                                  color_by="side",
                                  color_dict=side_color,
                                  default_marker="$\u2B24$",
                                  fig=img_fig, 
                                  ax=img_ax)

flash_fig, flash_ax = plot_loc_unicode(transformed_flash, 
                                      gradient_by="tick", 
                                      size=10,
                                      color_by="side",
                                      color_dict=side_color,
                                      marker_by="type",
                                      marker_dict=flash_marker,
                                      alpha=0.7,
                                      fig=loc_fig, 
                                      ax=loc_ax)

smoke_fig, smoke_ax = plot_loc_unicode(transformed_smoke, 
                                      gradient_by="start_tick", 
                                      size=50,
                                      default_color="Grays",
                                      default_marker='$\u2601$',
                                      alpha=0.02,
                                      fig=flash_fig, 
                                      ax=flash_ax)

plt.show()

In [15]:
kills_df.columns

Index(['tick', 'seconds', 'clock_time', 'attacker_steam_id', 'attacker_name',
       'attacker_team', 'attacker_side', 'attacker_x', 'attacker_y',
       'attacker_z', 'attacker_view_x', 'attacker_view_y', 'victim_steam_id',
       'victim_name', 'victim_team', 'victim_side', 'victim_x', 'victim_y',
       'victim_z', 'victim_view_x', 'victim_view_y', 'is_suicide',
       'is_teamkill', 'is_wallbang', 'penetrated_objects', 'is_first_kill',
       'is_headshot', 'victim_blinded', 'attacker_blinded', 'no_scope',
       'thru_smoke', 'distance', 'is_trade', 'weapon', 'weapon_class',
       'round_num', 'match_id', 'player_traded_name', 'player_traded_team',
       'player_traded_steam_id', 'assister_steam_id', 'assister_name',
       'assister_team', 'assister_side', 'flash_thrower_steam_id',
       'flash_thrower_name', 'flash_thrower_team', 'flash_thrower_side'],
      dtype='object')

In [39]:
round_kills = kills_df\
.groupby(["match_id","round_num"]).agg(
    ct_kills=("attacker_side",lambda side : (side == "CT").sum()),
    t_kills=("attacker_side",lambda side : (side == "T").sum())
)

/tmp/nix-shell.NBiiUK/ipykernel_1905/4266953083.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["match_id","round_num"]).agg(


In [29]:
rounds_df.columns

Index(['round_num', 'is_warmup', 'start_tick', 'freeze_time_end_tick',
       'end_tick', 'end_official_tick', 'bomb_plant_tick', 't_score',
       'ct_score', 'end_t_score', 'end_ct_score', 'ct_team', 't_team',
       'winning_side', 'winning_team', 'losing_team', 'round_end_reason',
       'ct_freeze_time_end_eq_val', 'ct_round_start_eq_val',
       'ct_round_spend_money', 'ct_buy_type', 't_freeze_time_end_eq_val',
       't_round_start_eq_val', 't_round_spend_money', 't_buy_type',
       'match_id'],
      dtype='object')

In [32]:
idx_r_df = rounds_df.set_index(["match_id","round_num"])
idx_r_df["n_ticks"] = idx_r_df["end_tick"] - idx_r_df["start_tick"]
idx_r_df = idx_r_df[["n_ticks","winning_side","round_end_reason",
                     "ct_round_spend_money","ct_buy_type",
                    "t_round_spend_money","t_buy_type"]]
idx_r_df.head(5)

n_ticks winning_side  \
match_id                             round_num                         
bfa9bbbe-bd16-4713-bd99-745378db05cb 1            11876            T   
                                     2            15789            T   
                                     3            21424            T   
                                     4            46594           CT   
                                     5            13771           CT   

                                               round_end_reason  \
match_id                             round_num                    
bfa9bbbe-bd16-4713-bd99-745378db05cb 1            TerroristsWin   
                                     2             TargetBombed   
                                     3             TargetBombed   
                                     4              TargetSaved   
                                     5                    CTWin   

                                                ct_round_spend_money  \
match_id                             round_num                         
bfa9bbbe-bd16-4713-bd99-745378db05cb 1                          3300   
                                     2                         10200   
                                     3                          1300   
                                     4                         26050   
                                     5                         17350   

                                               ct_buy_type  \
match_id                             round_num               
bfa9bbbe-bd16-4713-bd99-745378db05cb 1            Full Eco   
                                     2            Semi Buy   
                                     3            Semi Eco   
                                     4            Full Buy   
                                     5            Semi Buy   

                                                t_round_spend_money t_buy_type  
match_id                             round_num                                  
bfa9bbbe-bd16-4713-bd99-745378db05cb 1                         3550   Full Eco  
                                     2                        17000   Semi Buy  
                                     3                        11500   Full Buy  
                                     4                        13750   Full Buy  
                                     5                        15200   Full Buy

In [34]:
flashes_df.set_index(["match_id","round_num"])

tick    seconds clock_time  \
match_id                             round_num                                 
bfa9bbbe-bd16-4713-bd99-745378db05cb 2           17725  11.710938      01:44   
                                     2           18141  14.960938      01:41   
                                     2           19868  28.453125      01:27   
                                     2           20071  30.039062      01:25   
                                     2           20071  30.039062      01:25   
...                                                ...        ...        ...   
816e1612-9d2e-4893-af23-bda90c479598 16         251271   7.171875      00:33   
                                     16         251271   7.171875      00:33   
                                     17         261308  25.554688      01:30   
                                     17         261308  25.554688      01:30   
                                     17         264809  52.906250      01:03   

                                                attacker_steam_id  \
match_id                             round_num                      
bfa9bbbe-bd16-4713-bd99-745378db05cb 2          76561198010511021   
                                     2          76561198110563159   
                                     2          76561197979669175   
                                     2          76561198028458803   
                                     2          76561198028458803   
...                                                           ...   
816e1612-9d2e-4893-af23-bda90c479598 16         76561198010511021   
                                     16         76561198010511021   
                                     17         76561198045898864   
                                     17         76561198045898864   
                                     17         76561198040577200   

                                               attacker_name attacker_team  \
match_id                             round_num                               
bfa9bbbe-bd16-4713-bd99-745378db05cb 2                gla1ve      Astralis   
                                     2                Lucky       Astralis   
                                     2                k0nfig      Astralis   
                                     2                blameF      Astralis   
                                     2                blameF      Astralis   
...                                                      ...           ...   
816e1612-9d2e-4893-af23-bda90c479598 16               gla1ve      Astralis   
                                     16               gla1ve      Astralis   
                                     17              chopper   Team Spirit   
                                     17              chopper   Team Spirit   
                                     17                  sdy   Team Spirit   

                                               attacker_side   attacker_x  \
match_id                             round_num                              
bfa9bbbe-bd16-4713-bd99-745378db05cb 2                     T  -335.473602   
                                     2                     T  -540.563843   
                                     2                     T   178.267166   
                                     2                     T   203.921677   
                                     2                     T   203.921677   
...                                                      ...          ...   
816e1612-9d2e-4893-af23-bda90c479598 16                   CT  1000.038086   
                                     16                   CT  1000.038086   
                                     17                    T  -180.081177   
                                     17                    T  -180.081177   
                                     17                    T  -385.175140   

                                                 attacker_y  attacker_z  ...  \
match_id      

In [47]:
pd.concat([idx_r_df,round_kills],axis=1).dropna(how="all")

n_ticks winning_side  \
match_id                             round_num                         
bfa9bbbe-bd16-4713-bd99-745378db05cb 1          11876.0            T   
                                     2          15789.0            T   
                                     3          21424.0            T   
                                     4          46594.0           CT   
                                     5          13771.0           CT   
...                                                 ...          ...   
816e1612-9d2e-4893-af23-bda90c479598 13          8828.0           CT   
                                     14         13412.0           CT   
                                     15         10620.0           CT   
                                     16         45289.0           CT   
                                     17         15284.0            T   

                                               round_end_reason  \
match_id                             round_num                    
bfa9bbbe-bd16-4713-bd99-745378db05cb 1            TerroristsWin   
                                     2             TargetBombed   
                                     3             TargetBombed   
                                     4              TargetSaved   
                                     5                    CTWin   
...                                                         ...   
816e1612-9d2e-4893-af23-bda90c479598 13                   CTWin   
                                     14                   CTWin   
                                     15                   CTWin   
                                     16             BombDefused   
                                     17           TerroristsWin   

                                                ct_round_spend_money  \
match_id                             round_num                         
bfa9bbbe-bd16-4713-bd99-745378db05cb 1                        3300.0   
                                     2                       10200.0   
                                     3                        1300.0   
                                     4                       26050.0   
                                     5                       17350.0   
...                                                              ...   
816e1612-9d2e-4893-af23-bda90c479598 13                      22350.0   
                                     14                       9000.0   
                                     15                      12450.0   
                                     16                       3350.0   
                                     17                      19450.0   

                                               ct_buy_type  \
match_id                             round_num               
bfa9bbbe-bd16-4713-bd99-745378db05cb 1            Full Eco   
                                     2            Semi Buy   
                                     3            Semi Eco   
                                     4            Full Buy   
                                     5            Semi Buy   
...                                                    ...   
816e1612-9d2e-4893-af23-bda90c479598 13           Full Buy   
                                     14           Full Buy   
                                     15           Full Buy   
                                     16           Full Eco   
                                     17           Semi Buy   

                                                t_round_spend_money  \
match_id                             round_num                        
bfa9bbbe-bd16-4713-bd99-745378db05cb 1                       3550.0   
                                     2                      17000.0   
                                     3                      11500.0   
                                     4                      13750.0   
                                     5                      15200.0  

In [50]:
damages_df.columns

Index(['tick', 'seconds', 'clock_time', 'attacker_steam_id', 'attacker_name',
       'attacker_team', 'attacker_side', 'attacker_x', 'attacker_y',
       'attacker_z', 'attacker_view_x', 'attacker_view_y', 'attacker_strafe',
       'victim_steam_id', 'victim_name', 'victim_team', 'victim_side',
       'victim_x', 'victim_y', 'victim_z', 'victim_view_x', 'victim_view_y',
       'weapon', 'weapon_class', 'hp_damage', 'hp_damage_taken',
       'armor_damage', 'armor_damage_taken', 'hit_group', 'is_friendly_fire',
       'distance', 'zoom_level', 'round_num', 'match_id'],
      dtype='object')